In [1]:
# import of necessary libraries
import openeo
import logging
import json
from openeo.rest.imagecollectionclient import ImageCollectionClient

ModuleNotFoundError: No module named 'openeo'

In [ ]:
# define connetion parameters 
service_endpoint = "https://openeo.eurac.edu"
user = "guest"
password = "guest_123"

# authenticate with service endpoint
con = openeo.connect(service_endpoint)
con.authenticate_basic(user, password)
con.describe_account()

In [ ]:
# get some information about available functionality
cap = con.capabilities()
print(cap.version())
print(cap.list_features())
print(cap.currency())
print(cap.list_plans())

In [ ]:
# load a specific dataset
datacube = ImageCollectionClient.load_collection(session = con, collection_id = "S2_L2A_T32TPS_20M", bands = ['AOT', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B8A', 'B11', 'B12', 'SCL', 'VIS', 'WVP', 'CLD', 'SNW'])
# perform spatial subsetting (e.g around the city of Bolzano)
datacube = datacube.filter_bbox( west=11.279182434082033, south=46.464349400461145, east=11.406898498535158, north=46.522729291844286, crs="EPSG:32632")
# perform temporal subsetting (e.g. for the month of august in 2017)
temp = datacube.filter_temporal(extent=["2017-08-01T00:00:00Z", "2017-08-31T00:00:00Z"])
# map features of the dataset to variables (e.g. the red and near infrared band)
red = temp.band('B04')
nir = temp.band("B8A")
# perform operation using feature variables (e.g. calculation of NDVI (normalized difference vegetation index))
datacube = (nir - red) / (nir + red)
# reduce on temporal dimension with max operator
datacube = datacube.max_time()
# provide result as geotiff image
datacube = datacube.save_result(format="gtiff")

In [ ]:
# have a look at your process graph (not necessary and only for demonstration purposes)
print(json.dumps(datacube.graph,indent=2))

In [ ]:
# submit your process graph as new batch job to back-end
job = con.create_job(datacube.graph)

In [ ]:
# launch processing of submitted batch job
if job.job_id:
    print(job.job_id)
    print(job.start_job())
    print (job.describe_job())
else:
    print("Job ID is None")

In [ ]:
# obtain results and save to disk
if job.job_id:
    job.download_results("Sentinel2STfile.tiff")

In [ ]:
# Visualize your result
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import NoNorm
from matplotlib_scalebar.scalebar import ScaleBar
import numpy as np
from osgeo import gdal

%matplotlib inline

image_ds = gdal.Open("Sentinel2STfile.tiff")
ndvi_band = image_ds.GetRasterBand(1)
ndvi_image = ndvi_band.ReadAsArray()

dpi = 80
height, width = ndvi_image.shape

# What size does the figure need to be in inches to fit the image?
figsize = width / float(dpi), height / float(dpi)

fig = plt.figure(figsize=figsize)
ax = fig.add_axes([0, 0, 1, 1])

# Hide spines, ticks, etc.
ax.axis('on')

# Get geo extent using gdal
ext = image_ds.GetGeoTransform()
ncol = image_ds.RasterXSize
nrow = image_ds.RasterYSize
x_min = ext[0]
x_max = ext[0] + ext[1] * ncol
y_min = ext[3] + ext[5] * nrow
y_max = ext[3]

#plot map info elements
plt.xticks(np.arange(x_min, x_max, 1000))
plt.yticks(np.arange(y_min, y_max, 1000))
scalebar = ScaleBar(1, location='lower left', box_alpha=0.5)
plt.gca().add_artist(scalebar)
plt.arrow(x_min+1000,y_max-2000,0,900,fc="k", ec="k", linewidth = 4, head_width=200, head_length=500)
plt.text(x_min+950, y_max-500, 'N')
plt.title("NDVI form Sentinel-2")

# Display the image.
ax.imshow(ndvi_image, cmap='RdYlGn',vmin=-1,vmax=1, extent=[x_min, x_max, y_min, y_max])

